<a href="https://colab.research.google.com/github/Suji04/NormalizedNerd/blob/master/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##        Ce fichier va générer les données attendues par Spacy pour entrainement.







_N.B.: il s'agit ici du cas pour les ingrédients uniquement_

In [1]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import string

### Load Data

In [2]:
import pandas as pd

In [3]:
data= pd.read_json('./Data/jsonCorpusRecettes.json')

In [4]:
data.head()

,titre,preparation,ingredients
0,Poulet tandoori rouge,A préparer la veille !Mélanger l'épice Tandoor...,"[4 escalopes de poulet, 4 cuillères à soupe d'..."
1,Lasagnes végétariennes (facile),"Si vous utilisez des oignons, faites-les reven...","[6 tomates fraîches (ou pelées en boîte, à déf..."
2,Terrine de foie de porc,"- hacher (pas trop fin) le foie, le lard, les ...","[400 grammes de foie de porc , 200 grammes de ..."
3,Lapin au vin blanc (à la cocotte),Faire revenir les échalotes et les lardons dan...,"[1,8 kg de lapin (7 à 8 morceaux), 1 bouteille..."
4,Escalopes lucullus à la cocotte,"Coupez l'oignon en petits morceaux, faites rev...","[4 escalopes, 4 tranches de bacon, 4 tranches ..."


In [6]:
df = data.copy()
# Format du Dataframe
df.shape

(23071, 3)


- Chaque instruction (préparation) se compose de plusieurs phrases: on découpe en phrases.


In [9]:
import re, string

In [10]:
tmp=[]
for i in range(df.shape[0]):    
    tmp = tmp + re.split("[" +'!?.'+ "]+" , df.preparation[i])


In [11]:
tmpTitre=[]
for i in range(df.shape[0]):    
    tmpTitre = tmpTitre + re.split("[" +'!?.'+ "]+" , df.titre[i])

In [12]:
len(tmp),len(tmpTitre)

(247190, 23226)

=> On constate que toutes les recettes n'ont pas de titre.

On commence le prétraitement: en décollant et séparant les ponctuations dans un premier temps (pour la prépération mais aussi les titres) et en effaçant la casse (on passe tout les textes en minuscules).

In [13]:
listPhrases=[]
for i in range(len(tmp)):  
    if tmp[i] != '':
        listPhrases.append(tmp[i].replace('.',' . ').replace('!',' ! ').replace('?',' ? ').replace('\'','  ').replace('\n','').replace('(',' ( ').replace(')',' ) ').replace('"',' ').lower())


In [15]:
listPhrasesTitres=[]
for i in range(len(tmpTitre)):  
    if tmpTitre[i] != '':
        listPhrasesTitres.append(tmpTitre[i].replace('.',' . ').replace('!',' ! ').replace('?',' ? ').replace('\'','  ').replace('\n','').replace('(',' ( ').replace(')',' ) ').replace('"',' ').lower())


In [39]:
import json
import nltk
from nltk.stem.snowball import FrenchStemmer
import csv

In [21]:
# Méthode qui prend en entrée un fichier de stopwords, qui le charge et renvoie la liste des stopwords
def readJsonStopWordsInList(fileFrenchStopWords):
    dataDict={}
    with open(fileFrenchStopWords,encoding="utf8") as json_data:
        dataDict = json.load(json_data)
    return dataDict

In [22]:
# Normalisation, tokenisation, suppression des données non alphabetiques
def initialNormalizeAndTokenize(phrase_test):

    word_text =[]
    word_text = nltk.word_tokenize(phrase_test)

    # on peut enlever les ponctuations ce n est pas une analyse de sentiments.. 
    ponc ="!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~»«…\ '’"
    table = str.maketrans("","", ponc)

    # convert to lower case
    stripped = [w.translate(table).lower() for w in word_text]   
        
    # remove remaining tokens that are not alphabetic
    stripped = [word for word in stripped if word.isalpha()]
    
    assembled =" ".join(stripped)
    assembled = assembled.rstrip()
    word_text = nltk.word_tokenize(assembled)    
    return word_text

In [38]:
# Lemmatiseur (stemmer) en français
def initialLemmatizer(word_text):

    # lemmatizer = FrenchLefffLemmatizer()
    # lemWords = [lemmatizer.lemmatize(word) for word in word_text]
    stemmer = FrenchStemmer()
    lemWords = [stemmer.stem(word) for word in word_text]

    return lemWords 

In [25]:
# Méthode qui va prétraiter en utilisant le radical (stemmer) en particulier
def preprocessingUsingNorm(stopWords,text,adjWords):

    textNorm=[]
    for i,val in enumerate(text):
        phrase_test=val     

        # tokenization, suppression ponctuation et normalisation
        word_text = initialNormalizeAndTokenize(phrase_test)
    
        # stopWords supprimes et qq adjectifs et adverbes
        words = initialBow(stopWords,word_text,adjWords)
        
        
        # II) Lemmatisation (il s'agit de Stemmer ici mais c'est équivalent)
        lemWords = initialLemmatizer(words) 
        
        #stripped = unique(words)
        assembled =" ".join(lemWords)
        assembled = assembled.rstrip()
        textNorm.append(assembled)   
        
    return textNorm



In [26]:
# Méthode qui va supprimer les stopwords mais aussi une liste de mots (adjectifs, adverbes..)
def initialBow(stopWords,word_text,adjWords):
    tmp = [w for w in word_text if not w in stopWords]
    words = [w for w in tmp if not w in adjWords]
    return words

In [27]:
fileFrenchStopWords = "./Data/stopwords-fr.json"

In [28]:
# on charge les stopWords

stopWords={}
stopWords = readJsonStopWordsInList(fileFrenchStopWords) 


In [29]:
# liste de token à enlever de la liste également: ajectifs, adverbes...

df_Adj =  pd.read_csv('./Data/adjectifCulinaire.txt')

In [31]:
# On met en lowercase
for i in range(df_Adj.shape[0]):    
    df_Adj.Liste_A_Enlever[i] = df_Adj['Liste_A_Enlever'][i].lower()

In [32]:
df_Adj.head()

,Liste_A_Enlever
0,succulent
1,savoureux
2,aromatique
3,allégé
4,goûteux


In [34]:
adjWords =  df_Adj['Liste_A_Enlever'].values.tolist()

### On normalise , on enlève les stopwords, des ajectifs et adverbes, les nombres, on lemmatise (stemmer)

In [40]:
textNorm = preprocessingUsingNorm(stopWords,listPhrases,adjWords)  

In [41]:
len(textNorm)


230494

In [43]:
listPhrasesTitresNorm = preprocessingUsingNorm(stopWords,listPhrasesTitres,adjWords)  

In [45]:
# Récupération des données de sorties de la clusturisation: une liste d'ingrédients (radical)

df_ingredientsSave =  pd.read_csv('./Data/listIngredientsToSave.csv')

In [46]:
df_ingredientsSave.head()

,IngredientsSave
0,limoncello
1,épinard
2,combav
3,rapadur
4,mikados


In [47]:
df_ingredientsSave.shape

(1127, 1)

In [49]:
# transformation en liste
ingredients = df_ingredientsSave['IngredientsSave'].values.tolist()

## Spacy : EntityRuler et le label 'INGREDIENT' 

- Utilisation de _EntityRuler_ pour créer l'entité _'INGREDIENT'_ et faire le lien avec notre liste d'ingrédients.

In [50]:
import spacy
from spacy.language import Language
from spacy.pipeline import EntityRuler

In [51]:
nlp = spacy.load("fr_core_news_sm")

@Language.factory('ruler')
def Ruler(nlp, name):

    ingredient_pattern = [
        {"LOWER": {"IN": ingredients}}
    ]

    patterns = [{"label": "INGREDIENT", "pattern": ingredient_pattern}]
    return EntityRuler(nlp, patterns=patterns)

nlp.add_pipe('ruler', before="ner")


In [52]:
len(textNorm)

230494

In [55]:
# On crée nos data 80% train et 20% test: 184360 vs 46134
corpus = textNorm[0:184359]

In [56]:
TRAIN_DATA = []

#iterate over the corpus again and create spacy 2 data input (will be convert later)
for sentence in corpus:
    doc = nlp(sentence)
    
    #remember, entities needs to be a dictionary in index 1 of the list, so it needs to be an empty list
    entities = []
    
    #extract entities
    for ent in doc.ents:

        #appending to entities in the correct format
        entities.append([ent.start_char, ent.end_char, ent.label_])
        
    if len(entities) > 0:
        TRAIN_DATA.append([sentence, {"entities": entities}])
        


print (TRAIN_DATA[0])

['mélang épic tandoor épic indien roug lait yaourt grec huil jus citron', {'entities': [[7, 11, 'INGREDIENT'], [12, 19, 'INGREDIENT'], [20, 24, 'INGREDIENT'], [25, 31, 'INGREDIENT'], [37, 41, 'INGREDIENT'], [42, 48, 'INGREDIENT'], [49, 53, 'INGREDIENT'], [54, 58, 'INGREDIENT'], [59, 62, 'INGREDIENT'], [63, 69, 'INGREDIENT']]}]


In [59]:
print('Après traitement des données, la construction du data set donne: ',len(TRAIN_DATA),' données pour le dataTrain \n\n')

Après traitement des données, la construction du data set donne:  144443  données pour le dataTrain 




In [61]:
# On crée nos data 80% train et 20% test: 184360 vs 46134
corpusTest = textNorm[184360:230493]

In [62]:
TEST_DATA = []

#iterate over the corpus again
for sentence in corpusTest:
    doc = nlp(sentence)
    
    #remember, entities needs to be a dictionary in index 1 of the list, so it needs to be an empty list
    entities = []
    
    #extract entities
    for ent in doc.ents:

        #appending to entities in the correct format
        entities.append([ent.start_char, ent.end_char, ent.label_])
        
    if len(entities) > 0:
        TEST_DATA.append([sentence, {"entities": entities}])
        

print (TEST_DATA[0])

['mettr cuir moul savarin minut', {'entities': [[11, 15, 'INGREDIENT']]}]


In [63]:
print('Après traitement des données, la construction du data set donne: ',len(TEST_DATA),' données pour le dataTest \n\n')

Après traitement des données, la construction du data set donne:  35568  données pour le dataTest 




In [64]:
# Médthode pour sauvegarde de fichier json
def save_data(file,data):
    with open(file,"w",encoding='utf-8') as f:
        json.dump(data,f,indent=4)

In [69]:
corpusTitresTest = listPhrasesTitresNorm[0:23225]


23126

In [70]:
TEST_TITRE_DATA = []

#iterate over the corpus again
for sentence in corpusTitresTest:
    doc = nlp(sentence)
    
    #remember, entities needs to be a dictionary in index 1 of the list, so it needs to be an empty list
    entities = []
    
    #extract entities
    for ent in doc.ents:

        #appending to entities in the correct format
        entities.append([ent.start_char, ent.end_char, ent.label_])
        
    if len(entities) > 0:
        TEST_TITRE_DATA.append([sentence, {"entities": entities}])
        
    #TRAIN_DATA.append([sentence, {"entities": entities}])

print (TEST_TITRE_DATA[0])

['poulet tandoor roug', {'entities': [[0, 6, 'INGREDIENT'], [7, 14, 'INGREDIENT']]}]


In [72]:
print('Après traitement des données, la construction du data set donne: ',len(TEST_TITRE_DATA),' données pour le data d\'évaluation sur les titres\n\n')


Après traitement des données, la construction du data set donne:  21648  données pour le data d'évaluation sur les titres




## Sauvegarde des données pour entrainement: train/test/eval




In [73]:
save_data("./outputData/ingredients_training_data2.json",TRAIN_DATA)

In [74]:
save_data("./outputData/ingredients_test_data2.json",TEST_DATA)

In [75]:
save_data("./outputData/ingredientsTitre_test_data2.json",TEST_TITRE_DATA)

## Sauvegarde des données pour la partie préparation/instruction des recettes

In [85]:
df_Prep = pd.DataFrame({'Instructions': textNorm})
df_Prep

,Instructions
0,prépar veil
1,mélang épic tandoor épic indien roug lait yaou...
2,découp escalop poulet petit morceau enlev méla...
3,repos
4,fair grill morceau poêl rôt brochet serv sauc ...
...,...
230489,ajout crem maizen oignon ciboulet sal poivr
230490,répart carr émiet fond tart versezy prépar oeuf
230491,dispos tomat appui
230492,enfourn min


In [80]:
# On enregistre les données prétraitées pour la partie sur la section préparation une ligne correspond à une phrase
df_Prep.to_csv('./Data/recettesParPhrases2.txt',index=None)